In [1]:
import json
import os
import pandas as pd
import ast
import numpy as np

# 筛选出指定排名的类别

In [2]:
def select_class(ranks, save_path, imagenet_class_index_path='data/imagenet_class_index.json', chinese_class_index_path = 'data/chinese_class_index.json'):
    ''' 筛选出指定排名的类别
    
    Args:
    - ranks: list, 指定的排名
    - class_index_dict: dict, 类别索引
    - chinese_class_index_dict: dict, 中文类别索引
    - acc: dict, 所有类别准确率
    '''
    with open(imagenet_class_index_path) as f:
        class_index_dict = json.load(f)

    with open(chinese_class_index_path) as f:
        chinese_class_index_dict = json.load(f)
    
    acc = json.load(open(save_path))
    
    sorted_acc = sorted(acc.items(), key=lambda x: x[1], reverse=True)
    selected_items = {k: v for i, (k, v) in enumerate(sorted_acc) if i+1 in ranks}
    result = {}
    for rank, (class_code, accuracy) in zip(ranks, selected_items.items()):
        for id, (code, name) in class_index_dict.items():
            if code == class_code:
                chinese_name = chinese_class_index_dict[str(id)]
                corect_num = int(accuracy * 50)
                result[id] = [class_code, name, chinese_name, accuracy, corect_num, rank]
                break
    return result

In [3]:
save_path = 'data_stage2/class_acc_vit_b_16.json'
result = select_class([1, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], save_path)
print(result)

{'1': ['n01443537', 'goldfish', '丁鲷', 1.0, 50, 1], '512': ['n03109150', 'corkscrew', '敞篷车', 0.98, 49, 100], '569': ['n03417042', 'garbage_truck', '裘皮大衣', 0.96, 48, 200], '642': ['n03721384', 'marimba', '沙球（一种打击乐器）', 0.94, 47, 300], '959': ['n07831146', 'carbonara', '干草', 0.92, 46, 400], '680': ['n03825788', 'nipple', '项链', 0.88, 44, 500], '314': ['n02233338', 'cockroach', '竹节虫', 0.84, 42, 600], '468': ['n02930766', 'cab', '肉铺,肉菜市场', 0.8, 40, 700], '382': ['n02494079', 'squirrel_monkey', '蜘蛛猴', 0.74, 37, 800], '460': ['n02894605', 'breakwater', '奶罩', 0.66, 33, 900], '782': ['n04152593', 'screen', '记分牌', 0.22, 11, 1000]}


# 对单步法结果文件进行处理

In [4]:
id2rankdict = {}
id_list = result.keys()
for id in id_list:
    id2rankdict[int(id)] = result[id][-1]
id2rankdict

{1: 1,
 512: 100,
 569: 200,
 642: 300,
 959: 400,
 680: 500,
 314: 600,
 468: 700,
 382: 800,
 460: 900,
 782: 1000}

In [5]:
data = pd.read_excel('./data_stage2/classified_single_attack/classified_single_attack_1010.xlsx')
data.head()

,index,model,algo,mask_mode,parameter,eta,pixel_attacked,attack_ratio_per_channel,l1_norm,l2_norm,success_rate,original_loss,loss_dict_attacked,picture_all,picture_attacked,run_time,rank
0,1,vit_b_16,fgsm,positive,NaN,0.01,75266,"[0.4992, 0.5007, 0.5001]",44.2947,0.016710,0.24,-0.029,{0.01: -1.273},50,11,20.838,1
1,1,vit_b_16,fgsm,negative,NaN,0.01,75261,"[0.5008, 0.4993, 0.4999]",44.3118,0.016843,0.24,-0.029,{0.01: -1.275},50,11,21.150,1
2,1,vit_b_16,fgsm,all,NaN,0.01,150528,"[1.0, 1.0, 1.0]",88.6064,0.033553,0.30,-0.029,{0.01: -1.824},50,14,20.858,1
3,1,vit_b_16,fgsm,topr,0.15,0.01,22579,"[0.143, 0.1969, 0.1101]",47.4759,0.029320,0.18,-0.029,{0.01: -0.979},50,8,21.189,1
4,1,vit_b_16,fgsm,lowr,0.85,0.01,127948,"[0.857, 0.8031, 0.8899]",41.1295,0.004232,0.26,-0.029,{0.01: -1.373},50,12,21.484,1


In [6]:
def mean_of_lists(series):
    # 将每个元素转换为 NumPy 数组，然后计算均值
    arrays = np.array(series.tolist(), dtype=float)
    return np.mean(arrays, axis=0).tolist()

def onestep_data(data): 
    '''将多个batch的数据合并''' 
    data['parameter'] = data['parameter'].fillna(-1)
    data['attack_ratio_per_channel'] = data['attack_ratio_per_channel'].apply(ast.literal_eval)
    data['rank'] = data['index'].map(id2rankdict)
    data[['R', 'G', 'B']] = pd.DataFrame(data['attack_ratio_per_channel'].tolist(), index=data.index)
    data.drop('attack_ratio_per_channel', axis=1, inplace=True)
    return data

In [7]:
data = onestep_data(data)
data.head()

,index,model,algo,mask_mode,parameter,eta,pixel_attacked,l1_norm,l2_norm,success_rate,original_loss,loss_dict_attacked,picture_all,picture_attacked,run_time,rank,R,G,B
0,1,vit_b_16,fgsm,positive,-1.00,0.01,75266,44.2947,0.016710,0.24,-0.029,{0.01: -1.273},50,11,20.838,1,0.4992,0.5007,0.5001
1,1,vit_b_16,fgsm,negative,-1.00,0.01,75261,44.3118,0.016843,0.24,-0.029,{0.01: -1.275},50,11,21.150,1,0.5008,0.4993,0.4999
2,1,vit_b_16,fgsm,all,-1.00,0.01,150528,88.6064,0.033553,0.30,-0.029,{0.01: -1.824},50,14,20.858,1,1.0000,1.0000,1.0000
3,1,vit_b_16,fgsm,topr,0.15,0.01,22579,47.4759,0.029320,0.18,-0.029,{0.01: -0.979},50,8,21.189,1,0.1430,0.1969,0.1101
4,1,vit_b_16,fgsm,lowr,0.85,0.01,127948,41.1295,0.004232,0.26,-0.029,{0.01: -1.373},50,12,21.484,1,0.8570,0.8031,0.8899


In [8]:
data.to_excel('./data_stage2/classified_single_attack/classified_single_attack_1010.xlsx', index=False)

# 多步法结果处理

In [ ]:
data = pd.read_excel('./data_stage2/classified_multi_attack/classified_multi_attack_1011.xlsx')

In [ ]:
def multi_step_data(data): 
    '''将多个batch的数据合并''' 
    data['parameter'] = data['parameter'].fillna(-1)
    data['rank'] = data['index'].map(id2rankdict)
    return data

# 结果分析